# Première approche

## Objectif : Première prise en main d'Open fisca France. 

- Importer Openfisca
- Créer un cas type de base
- Calculer revenu disponible, irpp, etc
- Simuler une réforme


## Import d'openfisca

### Import des packages


In [16]:
from openfisca_france import FranceTaxBenefitSystem
import openfisca_core

## Création d'un scénario

### Import du système socio-fiscal français

In [2]:
tax_benefit_system = FranceTaxBenefitSystem()

In [3]:
scenario = tax_benefit_system.new_scenario()

### Création d'un cas type

On prend un cas type simple : une famille monoparentale avec deux enfants. On décrit, la période, le premier parent et chacun des enfants.

In [4]:
scenario.init_single_entity(
    period = '2017',
    parent1 = dict(
        age = 40,
        salaire_de_base = 50000,
        categorie_salarie = 'prive_cadre' 
        ),
    enfants = [
        dict(age = 1),
        dict(age = 10),
        ],
    )

## Simulation

In [5]:
simulation = scenario.new_simulation()

### Calcul de l'impot sur le revenu (IRPP)

In [6]:
simulation.calculate('irpp', '2017')[0].round()

-2380.0

### Calcul du revenu disponible

In [7]:
simulation.calculate('revenu_disponible', '2017')[0].round()

43361.0

In [8]:
simulation.calculate('revenu_net', '2017')[0].round()

39002.0

In [9]:
simulation.calculate('af', '2017-01')[0].round()

130.0

In [10]:
simulation.calculate('ppa', '2017-01')[0].round()

0.0

## Simulation d'une réforme simple

On simule une réforme de l'impôt sur le revenu. Par exemple, on passe le taux d'impôt sur la première tranche de 0,14 à 0,3.

In [ ]:
On importe les fonction Reform et period de openfisca-core. 

In [ ]:
from openfisca_core.reforms import Reform
from openfisca_core.periods import period

On regarde les taux sur la première tranche

In [11]:
tax_benefit_system.parameters.impot_revenu.bareme.brackets[1].rate

2014-01-01: 0.14
2006-01-01: 0.055
2003-01-01: 0.0683
2002-01-01: 0.0705
2001-01-01: 0.075
2000-01-01: 0.0825
1999-01-01: 0.095
1996-01-01: 0.105
1993-01-01: 0.12
1973-01-01: 0.05
1948-01-01: 0.1
1945-01-01: 0.12

On définit la réforme

In [26]:
def modify_parameters(parameters):
    reform_period = period("2017")
    parameters.impot_revenu.bareme.brackets[1].rate.update(period = reform_period, value = 0.30)
    return parameters

class modif_irpp(Reform):
    name = u'Increase rate of irpp'

    def apply(self):
        self.modify_parameters(modifier_function = modify_parameters)

On l'applique au système socio fiscal français

In [27]:
tax_benefit_system_reformed = modif_irpp(tax_benefit_system)

In [35]:
tax_benefit_system_reformed.parameters.impot_revenu.bareme.brackets[1].rate

2018-01-01: 0.14
2017-01-01: 0.3
2014-01-01: 0.14
2006-01-01: 0.055
2003-01-01: 0.0683
2002-01-01: 0.0705
2001-01-01: 0.075
2000-01-01: 0.0825
1999-01-01: 0.095
1996-01-01: 0.105
1993-01-01: 0.12
1973-01-01: 0.05
1948-01-01: 0.1
1945-01-01: 0.12

On définit un nouveau scénario

In [36]:
scenario2 = tax_benefit_system_reformed.new_scenario()

In [29]:
scenario2.init_single_entity(
    period = '2017',
    parent1 = dict(
        age = 40,
        salaire_de_base = 50000,
        categorie_salarie = 'prive_cadre' 
        ),
    enfants = [
        dict(age = 1),
        dict(age = 10),
        ],
    )

In [30]:
simulation2 = scenario2.new_simulation()

In [33]:
simulation2.calculate('irpp', '2017')[0].round()

-5100.0

In [34]:
simulation.calculate('irpp', '2017')[0].round()

-2380.0

L'impôt sur le revenu est passé de 2 380 euros à 5 100 euros.